In [ ]:
import numpy as np
from sklearn.manifold import MDS
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

# Make a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
patterns = LinkNodeGraphPatterns()
g         = patterns.createPattern('stanford_facebook_networks')
df        = patterns.nxGraphToPolarsDataFrame(g)

# Determine the distances
dists     = dict(nx.all_pairs_dijkstra_path_length(g))
nodes     = list(dists.keys())
node_to_i = {}
for i in range(len(nodes)): node_to_i[nodes[i]] = i
dissim = np.zeros((len(nodes),len(nodes)))
for _node0_ in g.nodes:
    for _node1_ in g.nodes:
        if _node0_ == _node1_:            continue
        if _node1_ not in dists[_node0_]: continue
        dissim[node_to_i[_node0_]][node_to_i[_node1_]] = dists[_node0_][_node1_]

# Execute MDS
mds           = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_transformed = mds.fit_transform(dissim)
pos_mds           = {}
for i in range(len(nodes)): pos_mds[nodes[i]] = [X_transformed[i][0], X_transformed[i][1]]

# Comparison Springs
pos_nx_spring     = nx.spring_layout(g)

# Color the nodes
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

_params_ = {'df':df, 'relationships':[('fm','to')], 'node_color':_node_colors_}
rt.tile([rt.link(pos=pos_mds,       **_params_),
         rt.link(pos=pos_nx_spring, **_params_)])